In [322]:
import os

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [323]:
# for i in range(n):
#     for prefix in ['x_train_split', 'y_train_split', 'y_onehot', 'index_split', 'model', 'history']:
#         var_name = f'{prefix}{i}'
#         if var_name in globals():
#             del globals()[var_name]

In [324]:
optimizer = "Adam"
loss_function = "categorical_crossentropy"
epochs = 60
batch_size = 128
learning_rate = .01
img_width = 32
img_height = 32
n = 5
num_class = 10
input_shape = (img_width, img_height, 3)
steps_per_epoch = 10000 // batch_size

In [325]:
#load CIFAR-10 DATA
def load_data():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    #normalize pixel values to 0-1 to simplify training
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    
    #flatten and encode labels
    y_train = y_train.flatten()
    y_test =  y_test.flatten()
    
    # y_train = to_categorical(y_train)
    # y_test = to_categorical(y_test)
    
    return x_train, y_train, x_test, y_test

In [326]:
x_train, y_train, x_test, y_test = load_data()

In [327]:
#AlexNet Model setup - includes matching parameters with MIAShield
def create_model(input_shape, num_class):
    model = Sequential()

    #Layer 1 - Conv2D with MaxPooling
    model.add(Conv2D(48, (3, 3), strides = (2, 2), activation = "relu", padding = "same", input_shape = input_shape))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    model.add(BatchNormalization())

    #Layer 2 - Conv2D with MaxPooling 
    model.add(Conv2D(96, (3, 3), activation = "relu", padding = "same"))
    model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
    model.add(BatchNormalization())

    #Layer 3 - Conv2D
    model.add(Conv2D(192, (3, 3), activation = "relu", padding = "same"))

    #Layer 4 - Conv2D
    model.add(Conv2D(192, (3, 3), activation = "relu", padding = "same"))

    #Layer 5 - Conv2D with MaxPooling
    model.add(Conv2D(256, (3, 3), activation = "relu", padding = "same"))
    model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
    model.add(BatchNormalization())

    #flatten
    model.add(Flatten())

    #Fully Connected layer 1
    model.add(Dense(512, activation = "relu"))
    model.add(Dropout(.50))
            
    #Fully Connected layer 2
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(.50))

    #Output layer
    model.add(Dense(num_class, activation = "softmax"))

    return model


In [328]:
X = x_train
Y = y_train
idx = np.arange(len(X))

skf = StratifiedKFold(n_splits = n, shuffle = True, random_state = 42)
skf.get_n_splits(X, Y)
    
for i, (_, test_index) in enumerate(skf.split(X,Y)):
    Xi = X[test_index]
    Yi = Y[test_index]
    Y_onehot = to_categorical(Yi, num_classes = num_class)
    index_split = idx[test_index]
    
    globals()[f'x_train_split{i}'] = Xi
    globals()[f'y_train_split{i}'] = Yi
    globals()[f'y_onehot{i}'] = Y_onehot
    globals()[f'index_split{i}'] = index_split

In [329]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range = .2,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [330]:
# train_generator = train_datagen.flow(
#     x_,
#     target_size=(img_width, img_height),
#     batch_size=batch_size,
#     class_mode='categorical'
# )

# val_generator = val_datagen.flow_from_directory(
#     y_test,
#     target_size=(img_width, img_height),
#     batch_size=batch_size,
#     class_mode='categorical'
# )

In [331]:
#recombine indices from across all splits
all_indices = np.concatenate([globals()[f'index_split{i}'] for i in range(n)])
print(f"Total number of indices: {len(all_indices)}")

# check for duplicate values to ensure disjointedness
has_duplicates = len(all_indices) != len(np.unique(all_indices))
print("Duplicate Indices Present:", has_duplicates)

# check to ensure full dataset is represented
covers_all = len(all_indices) == len(X)
print("Coverage of Dataset Achieved:", covers_all)

Total number of indices: 50000
Duplicate Indices Present: False
Coverage of Dataset Achieved: True


In [332]:
#verify that label distribution is consistent across all data partitions
for i in range(n):
    labels = globals()[f'y_train_split{i}']
    counts = Counter(labels)
    print(f"Fold {i} label distribution:", dict(counts))

Fold 0 label distribution: {7: 1000, 9: 1000, 4: 1000, 6: 1000, 5: 1000, 0: 1000, 3: 1000, 2: 1000, 1: 1000, 8: 1000}
Fold 1 label distribution: {6: 1000, 9: 1000, 4: 1000, 8: 1000, 7: 1000, 1: 1000, 5: 1000, 3: 1000, 0: 1000, 2: 1000}
Fold 2 label distribution: {9: 1000, 1: 1000, 3: 1000, 6: 1000, 4: 1000, 7: 1000, 2: 1000, 0: 1000, 8: 1000, 5: 1000}
Fold 3 label distribution: {1: 1000, 2: 1000, 3: 1000, 4: 1000, 0: 1000, 9: 1000, 5: 1000, 8: 1000, 6: 1000, 7: 1000}
Fold 4 label distribution: {7: 1000, 2: 1000, 0: 1000, 9: 1000, 3: 1000, 1: 1000, 8: 1000, 5: 1000, 4: 1000, 6: 1000}


In [333]:
for i in range(0, n):
    trained_model = create_model(input_shape, num_class)
    optimizer = tf.keras.optimizers.Adam(learning_rate=.01)
    loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    trained_model.compile(optimizer = optimizer, loss = loss_function, metrics = ['accuracy'])
    trained_model.summary()
    model_data = globals()[f'x_train_split{i}']
    target_data = globals()[f'y_train_split{i}']
    print(target_data.shape)
    datagen = train_datagen

    globals()[f'history{i}'] = trained_model.fit(datagen.flow(model_data, target_data, batch_size=128, shuffle=False),
                    steps_per_epoch = steps_per_epoch, epochs=epochs, validation_data=(x_test, y_test))
    globals()[f'model{i}'] = trained_model

C:\Users\Owner\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_115 (Conv2D)             │ (None, 16, 16, 48)     │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_69 (MaxPooling2D) │ (None, 8, 8, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_69          │ (None, 8, 8, 48)       │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_116 (Conv2D)             │ (None, 8, 8, 96)       │        41,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_70 (MaxPooling2D) │ (None, 3, 3, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_70          │ (None, 3, 3, 96)       │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_117 (Conv2D)             │ (None, 3, 3, 192)      │       166,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_118 (Conv2D)             │ (None, 3, 3, 192)      │       331,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_119 (Conv2D)             │ (None, 3, 3, 256)      │       442,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_71 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_71          │ (None, 1, 1, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_23 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_46 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_47 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,250,666 (4.77 MB)

 Trainable params: 1,249,866 (4.77 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_135 (Conv2D)             │ (None, 16, 16, 48)     │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_81 (MaxPooling2D) │ (None, 8, 8, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_81          │ (None, 8, 8, 48)       │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_136 (Conv2D)             │ (None, 8, 8, 96)       │        41,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_82 (MaxPooling2D) │ (None, 3, 3, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_82          │ (None, 3, 3, 96)       │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_137 (Conv2D)             │ (None, 3, 3, 192)      │       166,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_138 (Conv2D)             │ (None, 3, 3, 192)      │       331,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_139 (Conv2D)             │ (None, 3, 3, 256)      │       442,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_83 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_83          │ (None, 1, 1, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_27 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_54 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_55 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_83 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,250,666 (4.77 MB)

 Trainable params: 1,249,866 (4.77 MB)

 Non-trainable params: 800 (3.12 KB)

(10000,)
Epoch 1/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - accuracy: 0.1233 - loss: 2.7756 - val_accuracy: 0.1000 - val_loss: 2.3042
Epoch 2/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1562 - loss: 2.2961 - val_accuracy: 0.1000 - val_loss: 2.3042
Epoch 3/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.1348 - loss: 2.2509 - val_accuracy: 0.1000 - val_loss: 2.3037
Epoch 4/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.1406 - loss: 2.2287 - val_accuracy: 0.1000 - val_loss: 2.3038
Epoch 5/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - accuracy: 0.1728 - loss: 2.1530 - val_accuracy: 0.1000 - val_loss: 8.3861
Epoch 6/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1328 - loss: 2.1408 - val_accuracy: 0.1014 - val_loss: 3.6581
Epoch 7/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 71ms/step - accuracy: 0.1839 - loss: 2.0985 - val_accuracy: 0.1000 - val_loss: 2.3055
Epoch 8/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2031 - loss: 2.0986 - val_accuracy: 0

In [334]:
import os

save_dir = "models"
os.makedirs(save_dir, exist_ok = True)

for i in range(0, n):
    
    model = globals()[f'model{i}']
    
    model_path = os.path.join(save_dir, f'model_{i}.h5')
    model.save(model_path)
    print(f"Saved model {i} to {model_path}")


Saved model 0 to models\model_0.h5
Saved model 1 to models\model_1.h5
Saved model 2 to models\model_2.h5
Saved model 3 to models\model_3.h5
Saved model 4 to models\model_4.h5
